In [1]:
import pandas as pd
import numpy as np
import re

import time
import sys

import matplotlib.pyplot as plt
import seaborn as sns
from seaborn.widgets import color_palette

import folium
from folium.plugins import HeatMap
from folium import branca

## Concatenate datasets

In [2]:
# read csv
df_moscow = pd.read_csv('data/moscow_buildings_df.csv').drop(columns = ['Unnamed: 0'])
df_spb = pd.read_csv('data/spb_buildings_df.csv').drop(columns = ['Unnamed: 0'])
df_kazan = pd.read_csv('data/kazan_buildings_df.csv').drop(columns = ['Unnamed: 0'])

# add city name column 
df_moscow['city'] = 'moscow'
df_spb['city'] = 'spb'
df_kazan['city'] = 'kazan'

# concatenate datasets
df_concat = pd.concat([df_moscow, df_spb, df_kazan], ignore_index=True)
df_concat.to_csv('data/df.csv')

In [3]:
df_concat.sample(n=5)

,Address,Year,Flats,Entrance,Height,Square,Latitude,Longitude,city
34041,"ул. подольская, 7 литер а",1902,10,2,5,880.60,59.917015,30.322821,spb
52141,"ул. волгоградская, 9",1963,80,4,5,3668.70,55.829052,49.086157,kazan
15292,"ул. декабристов, 8 корпус 1",1984,237,5,12,13584.00,55.864460,37.599094,moscow
3842,"ул. 3-я песчаная, 5 корпус 4",1953,85,4,7,5821.40,55.794047,37.514563,moscow
44254,"ул. подвойского, 34 корпус 2 литер у",1971,118,6,5,5373.87,59.916685,30.486683,spb


## Functions for analysis 

### Statistic analysis visualization

In [18]:
def list_of_same_height_buildings_amount(df, name):
    height_list_a = df['Height'].value_counts()
    height_list_names = list(map(str, height_list_a.index))
    height_list_amount = height_list_a.values

    fig, ax = plt.subplots(figsize=(22, 8), dpi = 300)

    sns.barplot(x = height_list_names, y = height_list_amount,
                palette = 'Greens_r')
    plt.title(f'Different height buildings amount, {name}')
    plt.xlabel('Height, floors')
    plt.ylabel('Amount')
    plt.grid(axis = 'y', color = '#dce8dc')
    plt.savefig(f'pictures/{name}_plot_1.png')
    plt.close(fig)

In [19]:
def total_buildings_amount_by_years(df, name):
    years_list_a = df['Year'].value_counts()
    years_list_sorted = years_list_a.sort_index(ascending = True)
    total_years = years_list_sorted.index
    total_values = []
    # collect list of total buildings by every year
    for i in range(len(total_years)):
        if i == 0:
            total_values.append(years_list_sorted.values[i])
        else:
            el = total_values[i-1] + years_list_sorted.values[i]
            total_values.append(el)

    total_years_names = list(map(str, years_list_sorted.index))

    fig, ax = plt.subplots(figsize=(22, 8), dpi = 300)
    sns.barplot(x = total_years_names, y = total_values,
                color = '#dce8dd')
    sns.lineplot(x = total_years_names, y = total_values,
                color = 'green')
    plt.title(f'Total buildings amount, {name}')
    plt.xlabel('Year')
    plt.xticks(rotation=60)
    plt.ylabel('Amount')
    plt.grid(axis = 'y', color = '#dce2dd')
    # specify number of ticks on x-axis
    every_nth = 3
    for n, label in enumerate(ax.xaxis.get_ticklabels()):
        if n % every_nth != 0:
            label.set_visible(False)
    plt.savefig(f'pictures/{name}_plot_2.png')
    plt.close(fig)


In [20]:
def buildings_by_year_distribution(param_list, name):

    years_list_names = list(map(str, years_list_sorted.index))
    years_list_amount = years_list_sorted.values

    fig, ax = plt.subplots(figsize=(22, 8), dpi = 300)
    sns.barplot(x = years_list_names, y = years_list_amount,
                color = '#dce8dd')
    sns.lineplot(x = years_list_names, y = years_list_amount,
                color = 'green')
    plt.title(f'New buildings amount, {name}')
    plt.xlabel('Year of building')
    plt.xticks(rotation=60)
    plt.ylabel('Amount')
    plt.grid(axis = 'y', color = '#dce2dd')
    # specify number of ticks on x-axis
    every_nth = 3
    for n, label in enumerate(ax.xaxis.get_ticklabels()):
        if n % every_nth != 0:
            label.set_visible(False)
    plt.savefig(f'pictures/{name}_plot_3.png')
    plt.close(fig)


In [21]:
def square_by_year_distribution(param_list, name):

    square_years_list = []
    for year in years_list_names:
        sq_sum = sum(df.Square.loc[df['Year'] == int(year)])
        square_years_list.append(sq_sum)

    fig, ax = plt.subplots(figsize=(22, 8), dpi = 300)
    sns.barplot(x = years_list_names, y = square_years_list,
                color = '#dce8dd')
    sns.lineplot(x = years_list_names, y = square_years_list,
                color = 'green')
    plt.title(f'Built square meters amount, {name}')
    plt.xlabel('Year of building')
    plt.xticks(rotation=60)
    plt.ylabel('Square meters amount')
    plt.grid(axis = 'y', color = '#dce2dd')
    # specify number of ticks on x-axis
    every_nth = 3
    for n, label in enumerate(ax.xaxis.get_ticklabels()):
        if n % every_nth != 0:
            label.set_visible(False)
    plt.savefig(f'pictures/{name}_plot_4.png')
    plt.close(fig)

In [22]:
def average_apartment_square_by_year_distribution(param_list, name):

    flat_average_square_list = []
    for year in total_years_names:
        av_sq = sum(df.Square.loc[df['Year'] == int(year)]) / sum(df.Flats.loc[df['Year'] == int(year)])
        flat_average_square_list.append(av_sq)

    fig, ax = plt.subplots(figsize=(22, 8), dpi = 300)
    sns.barplot(x = years_list_names, y = flat_average_square_list,
                color = '#dce8dd')
    sns.lineplot(x = years_list_names, y = flat_average_square_list,
                color = 'green')
    plt.title(f'Apartment average square, {name}')
    plt.xlabel('Year of building')
    plt.xticks(rotation=60)
    plt.ylabel('Square, square meters')
    plt.grid(axis = 'y', color = '#dce2dd')
    # specify number of ticks on x-axis
    every_nth = 3
    for n, label in enumerate(ax.xaxis.get_ticklabels()):
        if n % every_nth != 0:
            label.set_visible(False)
    plt.savefig(f'pictures/{name}_plot_5.png')
    plt.close(fig)


In [23]:
def apartments_by_one_entrance_floor_count(df, param_list, name):

    # add new column with apartments per floor of enterance value for every address
    df['Flats_per_floor'] = df['Flats']/df['Entrance']/df['Height']


    flat_floor_list = []
    for year in total_years_names:
        fl_fl = sum(df.Flats_per_floor.loc[df['Year'] == int(year)]) / len(df.Flats_per_floor.loc[df['Year'] == int(year)])
        flat_floor_list.append(fl_fl)
    
    fig, ax = plt.subplots(figsize=(22, 8), dpi = 300)
    sns.barplot(x = years_list_names, y = flat_floor_list,
                color = '#dce8dd')
    sns.lineplot(x = years_list_names, y = flat_floor_list,
                color = 'green')
    plt.title(f'Apartment average amount by one entrance per floor, {name}')
    plt.xlabel('Year of building')
    plt.xticks(rotation=60)
    plt.ylabel('Apartment average amount')
    plt.grid(axis = 'y', color = '#dce2dd')
    # specify number of ticks on x-axis
    every_nth = 3
    for n, label in enumerate(ax.xaxis.get_ticklabels()):
        if n % every_nth != 0:
            label.set_visible(False)
    plt.savefig(f'pictures/{name}_plot_6.png')
    plt.close(fig)


In [24]:
def the_average_high_building_by_the_years(df, param_list, name):
    
    average_height_list = []

    for year in total_years_names:
        av_height = sum(df.Height.loc[df['Year'] == int(year)]) / len(df.Height.loc[df['Year'] == int(year)])
        average_height_list.append(av_height)

    fig, ax = plt.subplots(figsize=(22, 8), dpi = 300)
    sns.barplot(x = years_list_names, y = average_height_list,
                color = '#dce8dd')
    sns.lineplot(x = years_list_names, y = average_height_list,
                color = 'green')

    plt.title(f'Buildings average height, {name}')
    plt.xlabel('Year of building')
    plt.xticks(rotation=60)
    plt.ylabel('Height, floors')
    plt.grid(axis = 'y', color = '#dce2dd')
    # specify number of ticks on x-axis
    every_nth = 3
    for n, label in enumerate(ax.xaxis.get_ticklabels()):
        if n % every_nth != 0:
            label.set_visible(False)
    plt.savefig(f'pictures/{name}_plot_7.png')
    plt.close(fig)


In [25]:
def average_apartments_amount_in_one_building_for_every_year(df, param_list, name):
    average_flats_list = []
    for year in total_years_names:
        build_list = df.Flats[df.Year == int(year)].values
        average_flats_value = sum(build_list) / len(build_list)
        average_flats_list.append(average_flats_value)

    fig, ax = plt.subplots(figsize=(22, 8), dpi = 300)
    sns.barplot(x = years_list_names, y = average_flats_list,
                color = '#dce8dd')
    sns.lineplot(x = years_list_names, y = average_flats_list,
                color = 'green')
    plt.title(f'One building average apartments amount, {name}')
    plt.xlabel('Year of building')
    plt.xticks(rotation=60)
    plt.ylabel('Apartments amount')
    plt.grid(axis = 'y', color = '#dce2dd')
    # specify number of ticks on x-axis
    every_nth = 3
    for n, label in enumerate(ax.xaxis.get_ticklabels()):
        if n % every_nth != 0:
            label.set_visible(False)
    plt.savefig(f'pictures/{name}_plot_8.png')
    plt.close(fig)


In [31]:
def streets_count(df, param_list, name): 
    streets_list = []
    address_list = df.Address
    for text in address_list:
        street = text.split(',')[0]
        streets_list.append(street)

    df['Street'] = streets_list
    df_str_count = df['Street'].value_counts().tolist()
    df_str_names = df['Street'].value_counts().keys().tolist()


    top_25_str = df_str_names[:25]

    # list of all possible height 
    height_list_increase = sorted(list(map(int, height_list_names)))

    all_streets_height = []
    for n in top_25_str:
        # 1 d array for current street with 0 counters for every height
        height_street_l = list(range(len(height_list_increase)))
        height_street_list = [0 for el in height_street_l]

        # street name match in Address
        for h in range(len(df)):
            if n in df.Address[h]:
            # height match in Address
                for i in range(len(height_list_increase)):
                    if df.Height[h] == height_list_increase[i]:
                        height_street_list[i] +=1
        all_streets_height.append(height_street_list)

    fig, ax = plt.subplots(figsize=(24, 14), dpi = 300)
    sns.heatmap(all_streets_height, annot = True,
                square = True, 
                fmt = '2g',
                xticklabels = height_list_increase, 
                yticklabels = top_25_str, 
                cmap = 'Greens')
    plt.title(f'Different height buildings by streets (top 25), {name}')
    plt.xlabel('Height, floors')
    plt.ylabel('Street')
    plt.savefig(f'pictures/{name}_plot_9.png')
    plt.close(fig)

In [26]:
def streets_addresses_count(param_list1, param_list2, name):
    fig, ax = plt.subplots(figsize= (22, 18), dpi = 200)

    sns.barplot(x = df_str_names[:25], y = df_str_count[:25],
                palette = 'Greens_r')
    plt.title(f'On street buildings amount (top 25), {name}')
    plt.xlabel('Street')
    plt.xticks(rotation=75)
    plt.ylabel('Buildings amount')
    plt.grid(axis = 'y', color = '#dce8dc')
    plt.savefig(f'pictures/{name}_plot_10.png')
    plt.close(fig)


## Functions usage

In [32]:
%%time
df_read = pd.read_csv('data/df.csv').drop(columns = ['Unnamed: 0'])
cities_list = list(df_read.city.value_counts().index)
cities_center_coordinates = [[55.755042, 37.621589],    # moscow
                             [59.936984, 30.328483],    # spb
                             [55.795962, 49.105713]]    # kazan
for c in cities_list:
    df_base = df_read[df_read.city == c]
    print(f'Buildings amount in {c} = ', len(df_base))
    print(df_base.info())
    # clean base_df from unvalid data
    df = df_base[(df_base['Year'] > 800) &
             (df_base['Year'] < 2023) &
             (df_base['Flats'] < 10000) &
             (df_base['Entrance'] < 50) &
             (df_base['Height'] < 80) ]
    df.reset_index(drop= True , inplace= True )
    print(f'minimal year is {min(df.Year)}, maximum year is {max(df.Year)}')

    # functions
    list_of_same_height_buildings_amount(df, c)
    height_list_a = df['Height'].value_counts()
    height_list_names = list(map(str, height_list_a.index))
    total_buildings_amount_by_years(df, c)
    years_list_a = df['Year'].value_counts()
    years_list_sorted = years_list_a.sort_index(ascending = True)
    total_years_names = list(map(str, years_list_sorted.index))
    buildings_by_year_distribution(years_list_sorted, c)
    years_list_names = list(map(str, years_list_sorted.index))
    square_by_year_distribution(years_list_names, c)
    average_apartment_square_by_year_distribution(total_years_names, c)
    apartments_by_one_entrance_floor_count(df, total_years_names, c)
    the_average_high_building_by_the_years(df, total_years_names, c)
    average_apartments_amount_in_one_building_for_every_year(df, total_years_names, c)
    streets_count(df, height_list_names, c)
    df_str_count = df['Street'].value_counts().tolist()
    df_str_names = df['Street'].value_counts().keys().tolist()
    streets_addresses_count(df_str_count, df_str_names, c)

    # top strets df
    top_str_df = pd.DataFrame({'Street': df_str_names,
                            'Count': df_str_count})
    print(f'{len(top_str_df)} streets parsed\n' )
    # save dataframe as csv 
    df.to_csv(f'data/{c}_buildings_df_full.csv')

# read csv
df_moscow_full = pd.read_csv('data/moscow_buildings_df_full.csv').drop(columns = ['Unnamed: 0'])
df_spb_full = pd.read_csv('data/spb_buildings_df_full.csv').drop(columns = ['Unnamed: 0'])
df_kazan_full = pd.read_csv('data/kazan_buildings_df_full.csv').drop(columns = ['Unnamed: 0'])

# concatenate datasets
df_concat_full = pd.concat([df_moscow_full, df_spb_full, df_kazan_full], ignore_index=True)
df_concat_full.to_csv('data/df_full.csv')

Buildings amount in moscow =  31429
<class 'pandas.core.frame.DataFrame'>
Int64Index: 31429 entries, 0 to 31428
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Address    31429 non-null  object 
 1   Year       31429 non-null  int64  
 2   Flats      31429 non-null  int64  
 3   Entrance   31429 non-null  int64  
 4   Height     31429 non-null  int64  
 5   Square     31429 non-null  float64
 6   Latitude   31429 non-null  float64
 7   Longitude  31429 non-null  float64
 8   city       31429 non-null  object 
dtypes: float64(3), int64(4), object(2)
memory usage: 2.4+ MB
None
minimal year is 1600, maximum year is 2021


<ipython-input-23-02b9f353315a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Flats_per_floor'] = df['Flats']/df['Entrance']/df['Height']
<ipython-input-31-245faa334894>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Street'] = streets_list


2195 streets parsed

Buildings amount in spb =  16648
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16648 entries, 31429 to 48076
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Address    16648 non-null  object 
 1   Year       16648 non-null  int64  
 2   Flats      16648 non-null  int64  
 3   Entrance   16648 non-null  int64  
 4   Height     16648 non-null  int64  
 5   Square     16648 non-null  float64
 6   Latitude   16648 non-null  float64
 7   Longitude  16648 non-null  float64
 8   city       16648 non-null  object 
dtypes: float64(3), int64(4), object(2)
memory usage: 1.3+ MB
None
minimal year is 1728, maximum year is 2021


<ipython-input-23-02b9f353315a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Flats_per_floor'] = df['Flats']/df['Entrance']/df['Height']
<ipython-input-31-245faa334894>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Street'] = streets_list


1017 streets parsed

Buildings amount in kazan =  5507
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5507 entries, 48077 to 53583
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Address    5507 non-null   object 
 1   Year       5507 non-null   int64  
 2   Flats      5507 non-null   int64  
 3   Entrance   5507 non-null   int64  
 4   Height     5507 non-null   int64  
 5   Square     5507 non-null   float64
 6   Latitude   5507 non-null   float64
 7   Longitude  5507 non-null   float64
 8   city       5507 non-null   object 
dtypes: float64(3), int64(4), object(2)
memory usage: 430.2+ KB
None
minimal year is 1901, maximum year is 2021


<ipython-input-23-02b9f353315a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Flats_per_floor'] = df['Flats']/df['Entrance']/df['Height']
<ipython-input-31-245faa334894>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Street'] = streets_list


520 streets parsed

CPU times: user 1min 30s, sys: 4.93 s, total: 1min 35s
Wall time: 1min 33s


### Geo analysis visualization

In [38]:
def geo_year(df, coordianates, name):
    geo_year_param_list = list(map(list, zip(df['Latitude'], df['Longitude'], df['Year'])))
    map_year = folium.Map(location=coordianates, zoom_start=11, tiles= 'cartodb positron') # location is center

    HeatMap(geo_year_param_list, radius= 2, blur=2).add_to(map_year)
    colormap = branca.colormap.LinearColormap(['blue', 'lime', 'red'], 
                                            vmin = min(df.Year),
                                            vmax = max(df.Year),
                                            caption = 'Year of build')
    colormap.add_to(map_year)
    map_year.save(f'heatmaps/{name}_heatmap_year.html')

In [39]:
def geo_height(df, coordinates, name):
    geo_height_param_list = list(map(list, zip(df['Latitude'], df['Longitude'], df['Height'])))
    map_height = folium.Map(location=coordinates, zoom_start=11, tiles= 'cartodb positron') # location is center

    HeatMap(geo_height_param_list, radius= 2, blur=2).add_to(map_height)
    colormap = branca.colormap.LinearColormap(['blue', 'lime', 'red'], 
                                            vmin = min(df.Height),
                                            vmax = max(df.Height),
                                            caption = 'Height of building')
    colormap.add_to(map_height)
    map_height.save(f'heatmaps/{name}_heatmap_height.html')

In [40]:
geo_year(df_read[df_read.city == 'moscow'], cities_center_coordinates[0], 'moscow')
geo_height(df_read[df_read.city == 'moscow'], cities_center_coordinates[0], 'moscow')
geo_year(df_read[df_read.city == 'spb'], cities_center_coordinates[1], 'spb')
geo_height(df_read[df_read.city == 'spb'], cities_center_coordinates[1], 'spb')
geo_year(df_read[df_read.city == 'kazan'], cities_center_coordinates[2], 'kazan')
geo_height(df_read[df_read.city == 'kazan'], cities_center_coordinates[2], 'kazan')